In [ ]:
# importing module
import sys
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# appending a path
sys.path.append('/home/werner/workspace/Multigoal-Orchard-Drone-Planning-Library/cmake-build-release')

import pymgodpl

In [ ]:
rm = pymgodpl.load_robot_model(1.0)

In [ ]:
tms = pymgodpl.load_all_tree_meshes()

In [ ]:
success_rates = {}

for tm in tms:
    
    print('Working on :', tm.name)
    
    cd = pymgodpl.CollisionDetection(
        [tm.trunk], rm
    )

    N_SAMPLES = 1000

    collisions = pd.DataFrame(
        np.array([[not cd.collides(pymgodpl.sample_goal_region(fruit_position, random.randint(0,100000), rm)) 
                   for sample_i in range(N_SAMPLES)] 
                  for fruit_position in tm.fruit_positions()])
    )

    collisions.index.name = 'target'

    collision_tests = collisions.melt(var_name='sample_id', ignore_index=False).reset_index()
    sample_success_rate = collision_tests.groupby('target')['value'].agg(['mean','count'])

    # https://openstax.org/books/introductory-business-statistics/pages/8-3-a-confidence-interval-for-a-population-proportion
    sample_success_rate['err'] = 1.96 * np.sqrt((sample_success_rate['mean'] * (1.0 - sample_success_rate['mean']) / sample_success_rate['count']))
    
    success_rates[tm.name] = sample_success_rate

In [ ]:
for_tree = success_rates['appletree'].sort_values('mean').reset_index()

plt.plot(
    100.0 * for_tree.index / for_tree.index.max(),
    for_tree['mean']
)
plt.xlabel('% of apples')
plt.ylabel('% successful goal samples')
plt.grid()

In [ ]:
combined = pd.concat(success_rates, names=['tree'])
combined.index.name = 'tree'
combined = combined.reset_index()

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(combined, x='tree', y='mean')
plt.xticks(rotation=90)
plt.xlabel('Tree name')
plt.ylabel('Goal sample success rate')
plt.yscale('symlog', linthresh=0.1)

yticks = list(np.linspace(0,0.1,11)) + [0.15, 0.2,0.5,1.0]

plt.title('Distribution of difficulty (based on sampling success rate)')
plt.yticks(yticks, ['{:0.2f}'.format(t) for t in yticks])
plt.grid()
None